## USE CASE GENERATION USING LLM

In [21]:
import os
from langchain_openai import ChatOpenAI  # OpenAI integration
from langchain_core.prompts import PromptTemplate  # Core prompt handling
from langchain.chains import LLMChain

In [22]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Retrieve the OpenAI API key
openai_api_key = os.getenv("OPENAI_API_KEY")

# Set the API key in LangChain/OpenAI
os.environ["OPENAI_API_KEY"] = openai_api_key


### PROBLEM STATEMENT

In [25]:
problem_statement = """
An Airline Ticketing System allows the Clerk to order tickets and to look for a suitable flight. In order to
look up a ticket, the Clerk needs to enter the name of the traveler, the chosen flight, and finally booking
the airline tickets. When looking up a flight, the system provides a list of available flights between two airports
at a given date. There is also an option of multicity, where the customer chooses to fly through multiple
destinations with halting. The system must allow the ability to look up a flight without having to order a
ticket. When the order information to buy a ticket is entered, this information is automatically used to
look up the flight.
A third service (the help service) is available for the clerk while ordering a ticket to provide a help page
specific to ordering tickets. The help service can be invoked independently by the Clerk, which will provide
the Clerk with a “main” help page.
"""

### FEW SHOT EXAMPLE

In [23]:
use_case_few_shot_prompt = PromptTemplate(
    input_variables=["problem_statement"],
    template="""
Assume you are a software engineer developing requirements for an Airline Ticketing System.
Generate detailed standard textual use case models following the Requirement Documentation template.
Identify all the goals of the system and write a use case for each goal. 
For every use case, there will be one primary actor, and there can be one or multiple secondary actors.
An interaction step describes a base interaction step between the system and the environment (where the system must interact with at least one actor), and also describe alternatives for each of the base interaction steps.
Identify any exceptions that may exist in these interaction steps.

## FEW SHOT ##
### Example Use Case Format ###
Use Case: Order Ticket
Primary Actor: Clerk
Secondary Actors: Airline System, Payment Gateway
Preconditions: The Clerk must be logged into the system.
Steps:
1. Clerk enters passenger details.
2. Clerk selects the desired flight.
3. Clerk proceeds to booking.
4. System confirms ticket and generates PNR.

Exceptions:
- If the payment fails, prompt the user to retry.
- If the flight is full, suggest alternative flights.

Now generate any three use cases for the following problem statement:
{problem_statement}
"""
)

In [24]:
llm = ChatOpenAI(model_name="gpt-4", temperature=0.5)
use_case_chain = LLMChain(llm=llm, prompt=use_case_few_shot_prompt)

In [26]:
use_case_few_shot_output = use_case_chain.run(problem_statement=problem_statement)
print("\nGenerated Use Cases:\n")
print(use_case_few_shot_output)


Generated Use Cases:

### Use Case 1: Look Up Flight
Primary Actor: Clerk
Secondary Actors: Airline System
Preconditions: The Clerk must be logged into the system.
Steps:
1. Clerk enters the departure and arrival airports and the date.
2. System provides a list of available flights.
3. Clerk selects the desired flight to view more details.

Exceptions:
- If no flights are available, inform the Clerk and suggest alternative dates or airports.

### Use Case 2: Order Multicity Ticket
Primary Actor: Clerk
Secondary Actors: Airline System, Payment Gateway
Preconditions: The Clerk must be logged into the system.
Steps:
1. Clerk enters passenger details.
2. Clerk selects the "Multicity" option.
3. Clerk enters the desired flight details for each leg of the journey.
4. Clerk proceeds to booking.
5. System confirms ticket and generates PNR.

Exceptions:
- If the payment fails, prompt the user to retry.
- If any flight in the multicity journey is full, suggest alternative flights.

### Use Case

### CHAIN OF THOUGHT EXAMPLE

In [32]:
use_case_cot_prompt = PromptTemplate(
    input_variables=["problem_statement"],
    template="""
Assume you are a software engineer developing an Airline Ticketing System.
To ensure correctness, follow these steps:

1️⃣ **Identify all the goals** of the system.  
   Think carefully about what the system is trying to achieve and list all the goals.
   
2️⃣ For each goal, determine the **primary actor** (who initiates the action).  
   Think about who will trigger the action for each goal. It could be a customer, an admin, or a system.

3️⃣ Identify any **secondary actors** involved.  
   These are entities that play a supporting role, such as systems or external services.

4️⃣ Describe the **interaction steps** between actors and the system.  
   Detail the sequence of actions that take place to fulfill the goal.

5️⃣ Mention **exceptions** and alternative flows.  
   Consider possible exceptions or edge cases where the flow might differ from the norm.

### Example Use Case ###
Use Case: Order Ticket
Primary Actor: Clerk
Secondary Actors: Airline System, Payment Gateway
Preconditions: The Clerk must be logged into the system.

Steps:
1. Clerk enters passenger details.
2. Clerk selects the desired flight.
3. Airline System checks availability.
4. Clerk proceeds to booking.
5. System confirms booking and moves to payment.
6. Clerk enters payment details.
7. Payment Gateway processes payment.
8. Airline System confirms payment.
9. System generates PNR.
10. Confirmation message is sent to the Clerk or Customer.

Exceptions:
1. If payment fails, prompt the Clerk to retry.
2. If the flight is full, suggest alternative flights.
3. If the PNR cannot be generated, notify the Clerk of the issue.

Now, apply the same reasoning step-by-step to generate three use cases for:
{problem_statement}
"""
)


In [33]:
llm = ChatOpenAI(model_name="gpt-4", temperature=0.5)
use_case_chain = LLMChain(llm=llm, prompt=use_case_cot_prompt)

In [34]:
use_case_cot_output = use_case_chain.run(problem_statement=problem_statement)
print("\nGenerated Use Cases:\n")
print(use_case_cot_output)


Generated Use Cases:

Use Case 1: Look Up Flight
Primary Actor: Clerk
Secondary Actors: Airline System
Preconditions: The Clerk must be logged into the system.

Steps:

1. Clerk enters departure and arrival airports, and the date of travel.
2. Airline System provides a list of available flights.
3. Clerk selects the desired flight for further details.
4. Airline System provides flight details including availability, duration, and fare.
5. Clerk can choose to proceed to booking or look up another flight.

Exceptions:

1. If no flights are available, inform the Clerk.
2. If system error occurs while fetching flight details, notify the Clerk.

Use Case 2: Order Ticket with Multicity Option
Primary Actor: Clerk
Secondary Actors: Airline System, Payment Gateway
Preconditions: The Clerk must be logged into the system.

Steps:

1. Clerk enters passenger details.
2. Clerk selects the 'Multicity' option and enters the desired flights sequence.
3. Airline System checks availability for each fli

In [27]:
import re

def extract_actors_and_use_cases(text):
    actors = set()
    use_cases = []
    lines = text.split("\n")
    current_use_case = None
    
    for line in lines:
        line = line.strip()  # Remove extra spaces

        # Match "Use Case: <name>"
        use_case_match = re.match(r"Use Case:\s*(.+)", line)
        if use_case_match:
            current_use_case = use_case_match.group(1).strip()
            use_cases.append(current_use_case)
            continue  # Move to next line

        # Match "Primary Actor: <actor>"
        primary_actor_match = re.match(r"Primary Actor:\s*(.+)", line)
        if primary_actor_match:
            actors.add(primary_actor_match.group(1).strip())

        # Match "Secondary Actors: <actor1>, <actor2>"
        secondary_actors_match = re.match(r"Secondary Actors:\s*(.+)", line)
        if secondary_actors_match:
            sec_actors = secondary_actors_match.group(1).split(",")
            actors.update([actor.strip() for actor in sec_actors if actor.strip().lower() != "none"])

    return list(actors), use_cases

# Run extraction function
actors, use_cases = extract_actors_and_use_cases(use_case_output)

# Print extracted values for debugging
print("Extracted Actors:", actors)
print("Extracted Use Cases:", use_cases)


Extracted Actors: ['Payment Gateway', 'Clerk', 'Airline System', 'Help Service']
Extracted Use Cases: []
